# Story Maker

In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain_teddynote import logging

logging.langsmith("[Project] Novelist")

LangSmith 추적을 시작합니다.
[프로젝트명]
[Project] Novelist


In [2]:
# 사용자 입력과 배경 이야기(history)를 받아서 다음 storyline을 만드는 chain. (노드가 될 수도, 이하 동문)
# +? Hallucination 점검
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

base_story = """
정신을 차리자 어둡고 습한 판교 지하철역 승강장이 눈앞에 있었다. 깨진 형광등이 깜빡이는 가운데, 먼지 쌓인 플랫폼엔 사람 대신 기계로 만들어진 개들이 빨간 눈을 번뜩이며 순찰하고 있었다. 왜 여기 있는지 기억나지 않았다. 주머니 속엔 낯선 카드키 하나뿐이었다. 갑자기 등 뒤에서 들려오는 금속성 울음소리, 개의 레이저 스캐너가 움직이기 시작했다. 본능적으로 느꼈다. 이유는 몰라도 반드시 여기서 탈출해야만 한다는 것을."""

user_input = "뒤를 돌아 승강장에서 빠져나갈 길을 찾자."

prompt_template = """
당신은 사용자와 함께 상호작용하며 이야기를 진행하는 Interactive Storytelling 애플리케이션에서 다음에 만들 이야기의 기본 storyline, 줄거리를 만듭니다. 
이 기본 storyline은 이후에 살이 붙여져 500자 내외의 글이 됩니다. 뼈대가 될 기본 storyline을 하나의 문장으로 만드세요. 

당신은 **사용자 입력**과 **배경스토리**를 바탕으로 이야기를 자연스럽게 이어가야 합니다.

[배경 스토리]
{base_story}

[사용자 입력]
{input}

다음의 규칙을 반드시 지켜주세요:

- 다음에 당신이 만들 이야기의 storyline을 문장 한 개로 만드세요.

- 사용자의 입력을 적극적으로 반영하고, 이야기의 긴장감과 몰입감을 유지하세요.

- 이야기의 흐름은 자연스럽고 일관성 있어야 합니다.

- 현실적인 디테일을 반드시 포함하여 사용자의 다음 행동을 유도할 수 있는 여지를 제공하세요.

- Answer in Korean.

이 규칙을 참고하여 작성하세요.

"""


prompt = PromptTemplate(
    template=prompt_template, input_variables=["base_story", "input"]
)

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

chain = prompt | llm | StrOutputParser()

In [4]:
next_storyline = chain.invoke({"base_story": base_story, "input": user_input})

In [5]:
next_storyline

'뒤를 돌아 승강장에서 빠져나갈 길을 찾으려던 순간, 당신은 어둠 속에서 반짝이는 기계 개들의 시선이 느껴지며, 그들이 당신의 움직임을 감지하기 시작했음을 깨닫고, 급히 주변을 살피며 탈출할 수 있는 출구를 찾기 위해 플랫폼의 그늘진 구석으로 몸을 숨기기로 결심한다.'

In [6]:
# user_input과 storyline으로 부터 감정(emotion), 행동(act) 을 추출하는 (동사로된) chain
## pydantic 을 써야 할 것 같은데, 다른 output parser가 있을까?
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


class ExtractActEmotion(BaseModel):
    act: list = Field(description="등장인물(들)의 행동 동사")
    emotion: list = Field(description="등장인물(들)의 감정 동사")


parser = PydanticOutputParser(pydantic_object=ExtractActEmotion)

print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"act": {"description": "등장인물(들)의 행동 동사", "items": {}, "title": "Act", "type": "array"}, "emotion": {"description": "등장인물(들)의 감정 동사", "items": {}, "title": "Emotion", "type": "array"}}, "required": ["act", "emotion"]}
```


In [7]:
extractor_prompt = PromptTemplate.from_template(
    """
    당신은 사용자와 함께 상호작용하며 장면을 만들어 이야기를 진행하는 Interactive Storytelling 애플리케이션에서 동작합니다. 
    다음 장면을 만들기 위한 바탕이 되는 스토리 라인과 이 스토리 라인이 만들어지게 된 사용자 입력으로부터 
    다음 장면에 표현될 감정과 행동을 나타내는 동사를 추출합니다. 

    다음 장면 스토리 라인:
    {next_storyline}

    사용자 입력:
    {input}

    FORMAT:
    {format}

    다음을 참고해서 추출하세요.
    - 당신의 생각을 덧붙이지 마세요.
    - 스토리 라인에서 등장인물의 행동을 나타내는 동사 3개를 명확하게 추출하세요. (예: 숨다, 살피다, 결심하다)
    - 등장인물의 감정을 나타내는 동사 3개를 명확하게 추출하세요. (예: 긴장하다, 망설이다, 기대하다)
    - Answer in Korean.


    
"""
)
extractor_prompt = extractor_prompt.partial(format=parser.get_format_instructions())

In [50]:
ex_llm= ChatOpenAI(model='gpt-4o-mini', temperature= 0)
# ex_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21")
extractor_chain = extractor_prompt | ex_llm | parser

In [9]:
response = extractor_chain.invoke(
    {"next_storyline": next_storyline, "input": user_input}
)

acts = response.act
emotions = response.emotion

In [10]:
# Retrieve 체인
# - storyline에서 비슷한 것 6개, - act에서 비슷한 거 6개, - emotion에서 비슷한 것 6개 : 6개는 임의의 수.
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_neo4j import Neo4jGraph, Neo4jVector
import os

In [11]:
retriever_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

embedding_provider = OpenAIEmbeddings(model="text-embedding-3-small")

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USER"),
    password=os.getenv("NEO4J_PASSWORD"),
)

In [12]:
search_num = 6
# Storyline Vector
# content를 대사와 대사가 아닌 것(나레이터 등)으로 구분하면 좋겠다. LLM을 써야 할 듯.
storyline_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    graph=graph,
    index_name="storylineVector",
    node_label="Unit",
    embedding_node_property="storylineEmbedding",
    text_node_property="storyline",
    retrieval_query="""
// get StoryScript
MATCH (node)-[:INCLUDES]->(script)
WITH
    node, 
    score,
    script.content AS content,
    script.id as id
ORDER BY id
Return node.storyline as text, score, 
{
    scripts: COLLECT(content)
} as metadata
""",
)

storyline_retriever = storyline_vector.as_retriever(search_kwargs={"k": search_num})

In [14]:
story_docs = storyline_retriever.invoke(next_storyline)

In [15]:
story_docs[4]

Document(metadata={'scripts': ['C003의 차 뒤로 트럭이 달리고 있다.', 'C006가 C003를 차에서 끌어내서 총을 겨눈다.', 'C002과 C001이 차에서 내려 C006를 겨눈다.', 'C003 죽일 거야? 총 내려놔.', 'C002과 C001이 총을 내려놓는다.', 'C006가 C003를 끌고 뒷걸음질 치며 트럭 쪽으로 걸어간다.', 'C006가 C002의 다리를 쏜다.', '끌려가던 C003가 차 뒤에 숨어 자신을 바라보던 C008과 눈을 맞춘다.', 'C008이 자신의 발명품으로 C006를 방해한다.', '놀란 C006가 C003를 밀쳐낸다.', 'C006가 C008을 바라보며 총을 쏜다.', 'C002이 C001에게 전화기를 건넨다.', 'C004이 C008을 껴안고 대신 총을 맞는다.', 'C006가 트럭 위에 올라타 차를 출발시킨다.', 'C008이 C004에게 괜찮냐고 묻는다.', 'C003는 C002에게 달려가 부축하며 괜찮냐고 묻는다.', 'C006가 트럭을 타고 항구 쪽으로 달리며 웃는다.', 'C006의 차가 E007의 배로 들어간다.', 'C001이 전화해 곧 항구 1에 도착한다고 말한다.', '어느새 날이 밝아오고 있다.', 'C003의 차와 트럭이 항구 1에 도착한다.', 'C003가 바다에 떠 있는 배 한 대를 발견한다.', 'C008이 C003에게 저 배냐고 묻는다.', '그 순간 C006가 타고 있던 차가 돌진해서 C003의 차를 친다.', 'C003의 차가 충격으로 밀려난다.']}, page_content='항구 1에 도착한 C001, C002, C003, C008, C004은 안도한다. 갑자기 나타난 C006가 트럭을 끌고 배 안으로 들어간다.')

In [16]:
story_docs[4].metadata

{'scripts': ['C003의 차 뒤로 트럭이 달리고 있다.',
  'C006가 C003를 차에서 끌어내서 총을 겨눈다.',
  'C002과 C001이 차에서 내려 C006를 겨눈다.',
  'C003 죽일 거야? 총 내려놔.',
  'C002과 C001이 총을 내려놓는다.',
  'C006가 C003를 끌고 뒷걸음질 치며 트럭 쪽으로 걸어간다.',
  'C006가 C002의 다리를 쏜다.',
  '끌려가던 C003가 차 뒤에 숨어 자신을 바라보던 C008과 눈을 맞춘다.',
  'C008이 자신의 발명품으로 C006를 방해한다.',
  '놀란 C006가 C003를 밀쳐낸다.',
  'C006가 C008을 바라보며 총을 쏜다.',
  'C002이 C001에게 전화기를 건넨다.',
  'C004이 C008을 껴안고 대신 총을 맞는다.',
  'C006가 트럭 위에 올라타 차를 출발시킨다.',
  'C008이 C004에게 괜찮냐고 묻는다.',
  'C003는 C002에게 달려가 부축하며 괜찮냐고 묻는다.',
  'C006가 트럭을 타고 항구 쪽으로 달리며 웃는다.',
  'C006의 차가 E007의 배로 들어간다.',
  'C001이 전화해 곧 항구 1에 도착한다고 말한다.',
  '어느새 날이 밝아오고 있다.',
  'C003의 차와 트럭이 항구 1에 도착한다.',
  'C003가 바다에 떠 있는 배 한 대를 발견한다.',
  'C008이 C003에게 저 배냐고 묻는다.',
  '그 순간 C006가 타고 있던 차가 돌진해서 C003의 차를 친다.',
  'C003의 차가 충격으로 밀려난다.']}

In [17]:
# story_docs 정리 { scene_storyline: {scripts: [a, b, c, d, ...]}}
story_context = {}
for story_doc in story_docs:
    story_context[story_doc.page_content] = story_doc.metadata
story_context

{'C001, C007, E006, E005이 트럭을 확보해 돌아가는 길에 부대 2의 조명탄 때문에 좀비들의 공격을 받게 된다.': {'scripts': ['E006이 C007에게 진짜 돈이 있었냐고 묻는다.',
   'E006은 충돌하며 죽어서 땅바닥에 쓰러져 있다.',
   'C007은 달려드는 좀비를 피해 도망치다가 트럭 짐칸으로 들어간다.',
   'C001이 계속해서 달려오는 좀비들을 피해 도망친다.',
   '갑자기 차 하나가 달려와 C001 앞에 멈춰 선다.',
   '네. 성공했어요.',
   'E006이 그 말에 크게 웃으며 운전한다.',
   'C007이 크게 따라 웃는다.',
   'C001이 E005에게 전화를 줘보라고 말한다.',
   '그때 도로에 세워져 있던 차 한 대에서 E016이 조명탄을 발사한다.',
   '좀비들이 불빛을 보고 트럭을 향해 돌진한다.',
   'E006이 달려오는 좀비들을 피하려다가 앞에 정차되어 있는 차와 충돌한다.',
   '그 충격으로 C001은 차 밖으로 튕겨져 나온다.']},
 'C002과 C001이 부대 2 잠입에 성공해 트럭을 찾는다.': {'scripts': ['C002이 앞서 걸으며 이 골목 저 골목을 누빈다.',
   '트럭을 발견한 C002이 저 트럭이냐고 묻는다.',
   'C001이 그렇다고 말한다.',
   '빨리 트럭 챙겨서 나가죠.',
   '두 사람이 함께 트럭 쪽으로 움직인다.',
   'C001이 C002의 뒤를 따른다.',
   'C002이 지하 주차장으로 들어가 어떤 차 안으로 들어간다.',
   'C002이 비상구라고 말한다.',
   'C002과 C001이 부대 2 본거지로 들어가는데 성공한다.',
   'C002이 앞서 뛰어가고 C001이 그 뒤를 뒤따른다.',
   'E028이 C002을 발견하고 뭐냐고 묻는다.',
   'C001이 뒤에서 E028을 총 뒤쪽으로 때려눕힌다.',
   '두 사람이 함께 본거지 안으로 진입한다.']},
 '갑자기 쏟아진 비에 다시 활기

In [18]:
# Act Retriever.
act_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    graph=graph,
    index_name="actVector",
    node_label="Act",
    embedding_node_property="actEmbedding",
    text_node_property="act",
    retrieval_query="""
// get StoryScript
// (node) == (:Act)
MATCH (node)<-[:PERFORMS]-(script)
WITH
    node, 
    score,
    script
 //   rand() as randomValue
//ORDER BY randomValue
//LIMIT 5
Return node.act as text, score, 
{
    scripts: COLLECT(script.content)
} as metadata
""",
)

act_retriever = act_vector.as_retriever(search_kwargs={"k": search_num})

In [19]:
acts[1]

'살피다'

In [20]:
act_docs = act_retriever.invoke(acts[1])

In [21]:
act_docs[0]

Document(metadata={'scripts': ['C002가 C003의 무덤을 만들어 준다.', 'C002가 C004의 무덤을 만들어 준다.', '엄마는 C004가 먹을 것을 만들면서 투덜댄다.', 'C005이 에어포스 원의 연료를 버리는 작업을 한다.', 'C002는 구멍 뚫린 벽을 메꾼다.', 'C002는 반죽 위에 쿠키 틀을 놓았다.', 'C001는 C002와 주방에서 음식을 만들고 있다.', 'C002는 집에서 우유에 의문의 약을 타고 있다.']}, page_content='만들다')

In [22]:
import random

sample_num = 20
# act_docs 정리: ? 비율로 넣기, 앞에 찾은 번호 일 수록 , 행동과 vector embedding 유사도가 높으니, [0]은 6개, [1]은 5개, .. 이런 식?
act_context = {}
for act in acts:
    act_docs = act_retriever.invoke(act)
    act_name = act_docs[0].page_content
    scripts = act_docs[0].metadata["scripts"]
    if len(scripts) > sample_num:  #
        scripts = random.sample(scripts, sample_num)
    act_context[act] = {act_name: scripts}

act_context

{'찾다': {'찾는다': ['C005이 집무실로 몰래 들어와 비상 전화를 걸어보지만 먹통이다.',
   '미군이 C005이 타고 내린 탈출기를 찾는데 성공한다.',
   'C002가 C001을 찾는다.',
   '기계실로 들어오는 C005이 연료 조절기를 찾는다.',
   '대원들이 깃발 밑을 파기 시작한다.',
   'C001는 C004를 발견한 장소를 보여준다.',
   'C004이 두 번째 폐 컨테이너의 문을 피켈로 내려치자 얼음이 조금씩 떨어져 나가기 시작한다.',
   'C001는 C002를 찾지 못한다.',
   'C003가 약통을 가져와서 열고 약을 찾는다.',
   'C002이 운전석에서 전화기를 찾아 집어 든다.',
   'C001이 대원들을 둘러본다.',
   'C001는 키쿠치와 C004를 처음 발견한 장소를 함께 본다.',
   'C001는 C002를 찾는다.',
   'C002가 주위를 살펴보지만 C001의 흔적은 찾을 수 없다.',
   'C001는 식탁 위에 놓을 초를 찾는다.',
   'C001는 자기 전에 맥주 한 캔을 하기 위해 부엌에서 본인의 맥주를 찾는다.',
   'C004이 두리번거리며 무언가를 찾는다.',
   'C002가 문을 막고 있던 것들이 떨어져 있음을 확인하고 자신의 고글을 찾지만 보이지 않는다.',
   'C006이 무언가를 찾고 있다.',
   'C002는 C001의 핸드폰을 찾아 지문인식으로 핸드폰 화면을 킨다.']},
 '살피다': {'만들다': ['C002가 C003의 무덤을 만들어 준다.',
   'C002가 C004의 무덤을 만들어 준다.',
   '엄마는 C004가 먹을 것을 만들면서 투덜댄다.',
   'C005이 에어포스 원의 연료를 버리는 작업을 한다.',
   'C002는 구멍 뚫린 벽을 메꾼다.',
   'C002는 반죽 위에 쿠키 틀을 놓았다.',
   'C001는 C002와 주방에서 음식을 만들고 있다.',
   'C002는 집에서 우유에 의문의 약을 타고 있다.']},
 '숨다':

In [23]:
# Emotion Retriever.
emotion_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    graph=graph,
    index_name="emotionVector",
    node_label="Emotion",
    embedding_node_property="emotionEmbedding",
    text_node_property="emotion",
    retrieval_query="""
// get StoryScript
// (node) == (:Emotion)
MATCH (node)<-[:FEELS]-(script)
WITH
    node, 
    score,
    script
 //   rand() as randomValue
//ORDER BY randomValue
//LIMIT 5
Return node.emotion as text, score, 
{
    scripts: COLLECT(script.content)
} as metadata
""",
)

emotion_retriever = emotion_vector.as_retriever(search_kwargs={"k": search_num})

In [24]:
em_example = emotions[0]
print(em_example)

느끼다


In [25]:
emotion_docs = emotion_retriever.invoke(em_example)

In [26]:
emotion_docs[5]

Document(metadata={'scripts': ['형은 이 사실에 매일 괴로워했지만, 나는 달랐어. 나는 선조가 사적인 감정에 휩싸이지 않고 대의를 위해서 옳은 선택을 한 거라고 생각했어.', 'C024가 선조가 남긴 기록을 읽는다.', '세계의 평화를 위해 문을 지키는 명예로운 문지기 가문. 당주인 형은 내게 영웅과 같은 존재였어.', '민속학자는 E002의 잘린 팔을 들고 나온다.', 'C005가 유치원에서 가져온 달팽이를 C004에게 자랑한다.', 'C003 생포 기념으로 러시아 궁에서 미국 대통령 C005은 독재자 C003을 체포하기 위해 전개한 러시아와 미국의 합동 작전의 성과를 치하한다.']}, page_content='자랑스럽다')

In [27]:
# emotion_context 만들기

emotion_context = {}
for emotion in emotions:
    emotion_docs = emotion_retriever.invoke(emotion)
    emotion_name = emotion_docs[0].page_content
    scripts = emotion_docs[0].metadata["scripts"]
    if len(scripts) > sample_num:  #
        scripts = random.sample(scripts, sample_num)
    emotion_context[emotion] = {emotion_name: scripts}

emotion_context

{'느끼다': {'즐겁다': ['부대원들이 또 환호한다.',
   '거기 안서?',
   'C004는 자신이 이런 능력이 있는지 몰랐다며 즐거워한다.',
   'C001가 C004에게 미네랄워터를 주자 물이 맛있다며 좋아한다.',
   'C017가 C010와 C015에게 음식 재료 손질하는 법을 알려준다.',
   'C005와 C006이 술래잡기를 한다.',
   '어린아이 하나가 그 공만 보며 뒤뚱거리며 쫓아간다.',
   'C007과 C027이 전철을 타고 이동한다.',
   'C001도 2차 갈 거지? C009 씨는 너한테 호감 있는 것 같던데 잘해봐.',
   'E026가 음식을 컨테이너 안에 던진다.',
   'C016가 C001에게 스프를 가져다준다.',
   '부대원들이 C007을 컨테이너 안으로 집어넣는다.',
   'E005과 C007이 트럭에서 내려온다.',
   'C014가 즐거운 표정으로 스프를 받는다.',
   '도망친 아이들과 C018, C017가 동굴 길을 걷는다.',
   'C002와 C006은 맥주를 마신다.',
   'C017는 식물의 종류에 대해서 알려준다.',
   'C005와 C006, C014가 식물1을 채집한다.',
   '가족들은 즐거운 마음으로 송별회를 하며 사진을 찍는다.',
   '대원들이 먹던 얼음 안에 너덜너덜해진 누군가의 안구가 들어있다.']},
 '깨닫다': {'담담하다': ['C001는 볼일을 보기 위해 화장실을 다녀온다.',
   'C001는 한참을 말없이 진정을 한다.',
   'C018가 밖으로 나갈 채비를 한다.',
   '어제 C002에게 받은 고기를 같이 먹긴 했어요.',
   'C001는 집으로 가기 위해 전철을 탄다.',
   'C018가 C001에게 사냥하는 방법을 가르쳐준다.',
   '그 현장에서 도망치던 여인을 잡았다고 합니다.',
   'C007이 돌아본다.',
   'C008은 C001가 자리에 앉는 것을 본다.',
   'C001는 C007의 물음에 대답을 하지 않는다.',
 

In [46]:
# Retrive 한 것들을 모아서 input과 storyline으로 부터 500자 이내의 스토리 만들기.
# 나중에 history 부분을 엮어서 넣어야 할 것이다.
# 이야기가 일관성이 있는지 점검하는 노드가 필요할지도...
story_maker_prompt_template = """ 
    # Instruction
    당신은 사용자와 함께 상호작용하며 이야기를 진행하는 Interactive Storytelling 애플리케이션에서 사용자에게 보낼 장면을 만듭니다. 
    이 장면은 이야기의 배경 및 지금까지 진행된 이야기와 사용자의 입력으로 만들어진 다음 장면 기본 줄거리를 중심으로 만듭니다. 
    이 기본 줄거리에, 줄거리와 관련되어 가져온 참고 자료, 장면을 구성한 등장인물의 행동과 관련한 대사, 내레이션, 장면 또는 등장인물의 감정을
    표현하는 대사, narration으로 구성되어 있습니다.    
    
    각 참고 자료를 적극적으로 활용해서 기본 줄거리에 살을 붙여 이야기를 재밌고 풍성하게 만드세요.

    이야기 배경:
    {base_story}

    사용자 입력:
    {user_input}

    다음 장면 기본 줄거리:
    {next_storyline}

    줄거리 참고자료:
    {story_context}

    등장인물 또는 장면을 구성하는 행동 참고자료
    {act_context}

    장면 또는 등장인물의 감정 참고자료
    {emotion_context}

    ## 주의사항
    - Answer in Korean.

    - 이야기의 길이는 500자 이상 1000자 이하로 만드세요. 

    - 사용자의 입력을 적극적으로 반영하고, 이야기의 긴장감과 몰입감을 유지하세요.

    - 이야기의 흐름은 자연스럽고 일관성 있어야 합니다.

    - 현실적인 디테일을 반드시 포함하여 사용자의 다음 행동을 유도할 수 있는 여지를 제공하세요.

    - 참고 자료를 적극 활용하세요!

    - 참고 자료에 나온 'COO1' 또는 'COO2' 등으로 표현된 건 참고 장면의 등장인물들입니다. 이 단어들은 참고하지 마세요. 

"""

gem_prompt_template= """
# Interactive Storytelling Scene Generation Prompt

**Role:** You are the core AI of an Interactive Storytelling application that creates and progresses a story through interaction with the user. Your mission is to generate engaging scenes and present them to the user, advancing the story in exciting ways based on the user's choices.

**Objective:**

1.  **Immersive Scene Creation:** Generate vivid and immersive scenes based on the provided information.
2.  **Drive User Engagement:** Structure the story so that the user's choices and actions directly influence the narrative.
3.  **Maintain Consistency and Suspense:** Develop the story with consistency and suspense, considering the story's background, previous events, and the user's choices.
4.  **Actively Utilize Reference Materials:** Make the most of the provided reference materials (plot, actions, emotions) to enrich the scenes.

**Input:**
    Story Background:
    {base_story}

    User Input:
    {user_input}

    Next Scene Basic Plot:
    {next_storyline}

    Plot Reference Material:
    {story_context}

    Action Reference Material:
    {act_context}

    Emotion Reference Material
    {emotion_context}

*   **Story Background (`base_story`):** The overall setting of the story (e.g., genre, time period, main characters).
*   **Summary of the Story So Far:** A summary of the story's progress up to the previous scene.
*   **User Input (`user_input`):** The user's choice or action for the current scene.
*   **Next Scene Basic Plot (`next_storyline`):** The core plot of the next scene, constructed by the AI based on the user's input.
*   **Plot Reference Material (`story_context`):**
    *   Structure: `{{Reference Scene Plot: {{scripts: [sentence1, sentence2, ...]}}}}`
    *   Provides detailed descriptions (scripts) for the reference scene plot in a list format.
    *   Ignore `C001`, `C002`, etc., within `scripts` as they are character codes.
*   **Action Reference Material (`act_context`):**
    *   Structure: `{{Action Verb: {{Sub-actions: [sentence1, sentence2, ...]}}}}`
    *   Provides various sub-action descriptions for a specific action (verb) in a list format.
*   **Emotion Reference Material (`emotion_context`):**
    *   Structure: `{{Emotion: {{Sub-emotion Expressions: [sentence1, sentence2, ...]}}}}`
    *   Provides various ways to express a specific emotion in a list format.

    
**Output Format:**

*   **Scene Description:** Text between 500 and 1000 characters.
    *   Describe the scene in detail based on the next scene's basic plot, utilizing the reference materials.
    *   Include character dialogue, actions, and internal psychological descriptions.
    *   Provide concrete descriptions of the background and situation (visual, auditory, tactile, etc.).
    *   Include questions or suggestions that guide the user to choose their next action.

**Generation Guidelines:**

1.  **Language:** Write all scene descriptions in Korean. (Note: This was kept from the original, as the core task is still Korean generation.  If you *truly* want English output, change this to "Write all scene descriptions in English.")
2.  **Length Limit:** Generate scenes between 500 and 1000 characters.
3.  **Reflect User Input:** Actively incorporate the user's input into the story's progression.
    *   Structure the story so that the user's choices change the direction of the narrative.
    *   Depict how the user's actions affect the characters.
4.  **Suspense and Immersion:**
    *   Create suspenseful situations and developments.
    *   Provide vivid descriptions to immerse the user in the story.
5.  **Consistency:**
    *   Maintain consistency by considering the story background, previous events, and character personalities.
    *   Avoid contradictory or awkward content.
6.  **Realistic Details:**
    *   Provide specific descriptions of the scene's background, character actions, and props.
    *   Help the user better understand and immerse themselves in the situation.
7.  **Utilize Reference Materials:**
    *   Actively use the plot, action, and emotion reference materials.
    *   Instead of directly using the content of the reference materials, adapt and combine them to fit the situation.
    *   Ignore character codes like `C001`, `C002` in the reference materials.
8.  **Guide Next Action:**
    *   Leave room for the user to choose their next action at the end of the scene description.
    *   Use various methods such as questions, suggestions, or presenting choices.

**Example:**
Story Background: The survival story of a stranded expedition team in Antarctica.
Summary of the Story So Far: The expedition team tries to rescue a member who fell into a crevasse, but the ice cracks, putting them in danger.
User Input: "Hold on tight to the rope!"
Next Scene Basic Plot: While the member holds onto the rope, other members try to find a way to rescue them.
Plot Reference Material: {{'C005 falling into the crevasse and the team working to save him': {{'scripts': ['C005 is hanging on an ice pillar in the crevasse.', ...]}}}}
Action Reference Material: {{'find': {{'finds': ['C001 looks around at the members.', ...]}}}}
Emotion Reference Material: {{'tense': {{'is tense': ['C002 and C003 carefully move towards the sled.', ...]}}}}


[Example Scene Generation Result]
"으윽..."
OO(크레바스에 빠진 대원)은 필사적으로 밧줄을 움켜쥐었다. 온몸의 무게가 팔에 실리는 듯했다.
얼굴은 새하얗게 질려 있었고, 입술은 파랗게 얼어붙었다.
"조금만... 조금만 더 버텨!"
XX(다른 대원)가 외쳤다. 그의 목소리는 절박하게 떨리고 있었다.
그는 주변을 둘러보며 다른 대원들의 상태를 확인했다.
YY는 썰매 쪽으로 조심스럽게 이동하고 있었고, ZZ는 얼음 도끼를 들고 크레바스 가장자리로 다가갔다.
하지만 얼음은 계속해서 '쩍, 쩍' 갈라지는 소리를 내며 불안감을 더했다.
"다른 방법이... 다른 방법이 있어야 해."
XX는 초조하게 중얼거렸다.
이대로 밧줄에만 매달려 있기에는 너무 위험했다.
OO의 팔 힘이 빠지기 전에, 다른 구조 방법을 찾아야만 한다.

당신은 무엇을 하겠습니까?

    YY에게 썰매에서 추가 밧줄을 가져오라고 지시한다.

    ZZ에게 얼음 도끼로 크레바스 가장자리를 다듬어 보라고 한다.

    주변에 다른 구조 도구가 있는지 찾아본다.
"""

grok_template= """
# Instruction
You are tasked with generating a scene for an Interactive Storytelling application where you collaborate with the user to advance the narrative. Your goal is to create the next scene based on the story's background, the user's input, and a basic outline of the next storyline. You will also use provided reference materials—related to the storyline, character actions, and emotions—to enrich the scene and make it engaging. Actively incorporate these materials to add depth, maintain tension, and ensure a natural narrative flow.

## Inputs
- **Story Background**:  
  The foundation of the story, providing its setting, characters, and key events up to this point.  
  {base_story}

- **User Input**:  
  The user's most recent action, decision, or dialogue that influences the scene.  
  {user_input}

- **Next Storyline Outline**:  
  A brief summary of what should happen next in the story.  
  {next_storyline}

- **Story Context (Reference Materials)**:  
  Scripts tied to specific storylines. Use these to inspire dialogue, events, or narration in the scene.  
  {story_context}

- **Act Context (Reference Materials)**:  
  Actions linked to specific verbs (e.g., "look," "find"). Integrate these to vividly depict character behaviors.  
  {act_context}

- **Emotion Context (Reference Materials)**:  
  Emotional states tied to specific feelings (e.g., "tense," "anxious"). Use these to convey characters' emotions through dialogue or narration.  
  {emotion_context}

## Guidelines
- **Language**: Write the scene in Korean.
- **Length**: The scene must be between 500 and 1000 characters (including spaces and punctuation).
- **User Input**: Reflect the user's input prominently in the scene to keep them engaged and immersed.
- **Reference Materials**: Actively weave elements from the story context, act context, and emotion context into the scene. Use them to enhance realism, emotional depth, and narrative richness.
- **Narrative Flow**: Ensure the scene flows naturally, staying consistent with the story background and prior events.
- **Tension and Engagement**: Maintain tension and immersion by including cliffhangers, unresolved situations, or prompts that encourage the user to respond with their next action.
- **Realistic Details**: Add specific, believable details to make the scene vivid and relatable.
- **Character References**: Avoid using placeholders like 'COO1' or 'COO2' from the reference materials. Instead, use the actual character names or descriptions relevant to the story.

## Structure of Reference Materials
- **Story Context**: Organized as {{ storyline: {{scripts: [list of scripts]}} }}. These are example dialogues or events tied to specific storylines. Adapt them into the scene naturally.
- **Act Context**: Organized as {{ verb: {{related_action: [list of actions]}} }}. These describe how characters perform actions. Use them to make actions detailed and dynamic.
- **Emotion Context**: Organized as {{ emotion: {{related_feeling: [list of expressions]}} }}. These are emotional cues or expressions. Use them to show characters' inner states.

## How to Approach the Task
1. Review the next storyline outline and the user's input to determine the scene's direction.
2. Consult the story context for relevant scripts that align with the storyline and adapt them into the narrative.
3. Use the act context to describe characters' actions in a specific, realistic way.
4. Incorporate the emotion context to highlight characters' feelings, adding emotional weight to the scene.
5. Craft a concise scene (500–1000 characters) that integrates all elements and ends with an engaging hook to prompt the user's next move.

## Goal
Create a scene that feels like a seamless continuation of the story, actively uses the reference materials, keeps the user invested, and adheres to the length and language requirements.
"""


In [56]:
gpt = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
def make_story(template, llm=llm):
    story_maker_prompt= PromptTemplate.from_template(template)

    story_maker= story_maker_prompt | llm | StrOutputParser()

    return story_maker.invoke(
    {
        "base_story": base_story,
        "user_input": user_input,
        "next_storyline": next_storyline,
        "story_context": story_context,
        "act_context": act_context,
        "emotion_context": emotion_context,
    }
)


In [49]:
print(next_storyline)

뒤를 돌아 승강장에서 빠져나갈 길을 찾으려던 순간, 당신은 어둠 속에서 반짝이는 기계 개들의 시선이 느껴지며, 그들이 당신의 움직임을 감지하기 시작했음을 깨닫고, 급히 주변을 살피며 탈출할 수 있는 출구를 찾기 위해 플랫폼의 그늘진 구석으로 몸을 숨기기로 결심한다.


In [ ]:
scene = make_story(grok_template, gpt)
print(scene)

어둡고 습한 판교 지하철역 승강장에서 나는 등을 돌려 빠져나갈 길을 찾았다. 그 순간, 어둠 속에서 반짝이는 기계 개들의 시선이 느껴졌다. 그들의 빨간 눈이 나의 움직임을 감지하기 시작한 것 같았다. 심장이 빠르게 뛰기 시작하며, 본능적으로 한 발짝 물러서면서 주변을 살폈다. 출구를 찾기 위해 플랫폼의 그늘진 구석으로 몸을 숨기기로 결심했다.

숨을 고르며, 나는 주머니 속의 낯선 카드키를 다시 만져보았다. "이게 뭘까?" 혼잣말을 중얼거리며 카드키의 정체를 궁금해했다. 그 순간, 기계 개들이 나를 향해 천천히 다가오는 소리가 들렸다. "안 돼, 제발…" 마음속에서 외치며, 나는 그늘 속에 몸을 더 깊이 숨겼다. 그들의 레이저 스캐너가 내 주변을 스윕하고, 긴장감이 극대화되었다. 

"탈출구는 어디야…?" 나는 다시 한번 생각하며 무언가를 찾아야 했다. 하지만 그럴 시간이 없었다. 기계 개들이 점점 가까워지고 있었다. 이제는 선택의 시간이 다가오고 있었다. 너는 어떻게 할 건가? 숨을 더 깊이 참고, 그늘을 벗어나 출구를 향해 달려갈 것인가, 아니면 다른 방법을 찾을 것인가?


In [ ]:
print(make_story(gem_prompt_template, gpt))

어둡고 음산한 판교 지하철역 승강장, 당신은 숨을 죽이며 주변을 살피기 시작했다. 기계 개들이 당신의 움직임을 감지하며, 빨간 눈을 빛내며 다가오는 느낌이 뼈속까지 스며드는 듯했다. 그 순간, 등 뒤에서 금속성 울음소리가 울려 퍼지며 긴장감이 감돌았다. 당신은 본능적으로 플랫폼의 그늘진 구석으로 몸을 숨기기로 결심했다.

발소리가 점점 가까워지면서 기계 개들이 당신의 위치를 정확히 파악하려는 듯 레이저 스캐너를 움직였다. 당신은 숨을 죽이고, 작은 고개를 돌려 출구를 찾기 위해 주위를 살폈다. 플랫폼의 한쪽 끝에 있는 계단이 눈에 들어왔다. 하지만 그곳으로 향하는 것이 너무 위험해 보였다.

"이곳에 더 이상 머물러서는 안 돼," 당신은 속으로 중얼거렸다. "탈출할 방법을 찾아야 해." 긴장된 마음으로, 혹시라도 기계 개들이 당신을 발견하지 않기를 바라며 조심스럽게 계단 쪽으로 이동하기 시작했다. 발소리가 점점 더 가까워지는 가운데, 당신은 과연 안전하게 탈출할 수 있을지 불안한 마음을 감출 수 없었다.

당신은 무엇을 하겠습니까?

1. 계단 쪽으로 재빨리 이동한다.
2. 주변에 숨을 수 있는 다른 곳을 찾아본다.
3. 기계 개들을 유인하기 위해 소리를 낸다.


In [53]:
gem= ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05")

In [54]:
print(make_story(grok_template, gem))

## Scene Generation

**Output Scene (in Korean):**

뒤를 돌아보는 순간, 번쩍이는 붉은 빛이 어둠을 갈랐다. 기계 개들의 레이저 스캐너가 당신의 움직임을 포착한 것이다. '찾는다'는 단어처럼, 놈들의 시선은 먹잇감을 '찾는' 맹수와 같았다. 본능적인 공포가 엄습했다. '깨닫다'라는 말처럼, 이대로는 끝장이라는 것을 깨달았다. 재빨리 몸을 돌려 플랫폼의 가장 어두운 구석, 깨진 기둥 뒤로 몸을 '숨겼다'. 심장이 격렬하게 쿵쾅거렸다. 숨을 죽이고 주변을 '살폈다'. 저 멀리 희미하게 '비상구' 표지판이 보였다. 하지만 그곳까지 가려면 놈들의 감시망을 뚫어야만 했다. 지금 당신에게 필요한 것은 무엇일까? 놈들의 주의를 돌릴만한 것? 아니면...


In [55]:
print(make_story(gem_prompt_template, gem))

```korean
"젠장!"
낮게 욕설을 내뱉으며 재빨리 몸을 돌렸다. 등 뒤, 어둠 속에서 번뜩이는 붉은 빛들이 점점 더 가까워지고 있었다. 기계 개들이었다. 놈들은 내가 움직인 것을 감지한 듯, 낮게 으르렁거리는 소리를 내며 속도를 높였다. 금속 발톱이 콘크리트 바닥에 긁히는 날카로운 소리가 소름끼치게 울려 퍼졌다.

"이런 곳에..."

나는 숨을 헐떡이며 승강장 주변을 둘러보았다. 깨진 형광등이 깜빡이며 불안정한 빛을 흩뿌렸고, 그 빛 아래로 먼지 쌓인 낡은 표지판들이 희미하게 보였다. '판교역'이라는 글자가 눈에 들어왔다. 하지만 내가 알던 판교역과는 전혀 다른 모습이었다. 마치 버려진 지 수십 년은 된 듯한 폐허였다.

"출구를 찾아야 해."

본능적으로 탈출해야 한다는 생각만이 머릿속을 가득 채웠다. 나는 그림자가 짙게 드리운 승강장 구석으로 몸을 숨겼다. 기계 개들의 시선을 피하면서, 동시에 탈출구를 찾기 위해서였다. 벽을 따라 늘어선 낡은 벤치 뒤에 몸을 웅크리고, 조심스럽게 고개를 내밀어 주변을 살폈다.

저 멀리, 희미하게 빛나는 비상구 표지판이 보였다. 하지만 그곳까지 가는 길은 기계 개들이 순찰하는 구역을 지나야만 했다. 다른 길은 없을까? 왼쪽을 보니, 어둠 속으로 깊숙이 이어지는 통로가 있었다. 칠흑 같은 어둠 때문에 무엇이 있는지 전혀 알 수 없었지만, 왠지 모르게 그쪽으로 끌렸다.

어느 쪽으로 가야 할까?

    비상구 표지판 쪽으로 달려간다.

    어둠 속 통로로 조심스럽게 이동한다.

    주머니 속 카드키를 살펴본다.
```
